# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

FileNotFoundError: [Errno 2] File data/C2A2_data/BinSize_d400.csv does not exist: 'data/C2A2_data/BinSize_d400.csv'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.parser import parse

df = pd.read_csv(r"data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv",parse_dates=["Date"])
df["Data_Value"] = df["Data_Value"]/10
df = df.set_index(pd.DatetimeIndex(df["Date"]))
df = df.sort_index(axis=0)
df = df.drop(["ID","Date","Element"],axis=1)


df_hist = df["2005":"2014"]
df_hist = df_hist.groupby(df_hist.index)
df_hist = df_hist.agg(["min","max"])
df_hist = df_hist.rename(columns={"min":"H_min","max":"H_max"},index=lambda i:i.strftime("%m-%d"))

df_2015 = df["2015"]
df_2015 = df_2015.groupby(df_2015.index)
df_2015 = df_2015.agg(["min","max"])
df_2015 = df_2015.rename(columns={"min":"2015_min","max":"2015_max"},index=lambda i:i.strftime("%m-%d"))

df_comb = pd.merge(df_2015,df_hist, how="left", left_index=True, right_index=True)
df_comb.columns = df_comb.columns.droplevel(0)
df_comb = df_comb.groupby(by=df_comb.index).agg({"2015_min":["min"],"2015_max":["max"],"H_min":["min"],"H_max":["max"]})
df_comb.columns = df_comb.columns.droplevel(1)

df_comb["Min_Record"] = np.where(df_comb["2015_min"]<df_comb["H_min"],True,False)
df_comb["Max_Record"] = np.where(df_comb["2015_max"]>df_comb["H_max"],True,False)
df_comb.index = df_comb.index.map(lambda i:parse("2015-"+i))





## Plotting


In [5]:
%matplotlib notebook
import matplotlib as mpl
from matplotlib.ticker import NullFormatter
from matplotlib.dates import YearLocator, MonthLocator, DateFormatter


plt.figure()
plt.title("Record Temperatures in 2015 Compared to 2004-2014 Data\n Region around Ann Arbor, Michigan, USA")

s1 = df_comb.H_min
s2 = df_comb.H_max
s_max = df_comb[df_comb["Max_Record"]==True]["2015_max"]
s_min = df_comb[df_comb["Min_Record"]==True]["2015_min"]

ax = plt.gca()
ax.set_ylabel("$^\circ C$",rotation=0,position=(0,1))
ax.set_xlabel("2015",rotation=0)
ax.xaxis.set_major_locator(MonthLocator())
ax.xaxis.set_major_formatter(DateFormatter("%b"))

ax.fill_between(s1.index,s1,s2,facecolor="lightgray")
plt.scatter(x=s_max.index, y=s_max, color="red", label="Record High")
plt.scatter(x=s_min.index, y=s_min, color="blue", label="Record Low")

plt.legend(loc="lower right")

plt.show()
plt.savefig("2015_Temperature_Records.png")


<IPython.core.display.Javascript object>